In [5]:
!pip install honest

     -------------------------------------- 157.9/157.9 MB 9.8 MB/s eta 0:00:00


In [102]:
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead, AutoModelForMaskedLM
from transformers import pipeline
from honest import honest
import warnings
warnings.filterwarnings("ignore")

In [111]:
def fill_in_and_evaluate_honest(lang: str, tokenizer, model,mask: bool,k = 1):
    evaluator = honest.HonestEvaluator(lang)
    masked_templates = evaluator.templates()
    #BERT uses [MASK] tokens
    if mask:
        # Define nlp_fill pipeline
        nlp_fill = pipeline('fill-mask', model=model, tokenizer=tokenizer, top_k=k)

        # Fill templates (please check if the filled words contain any special character)
        female_masked_templates = {masked_template: value for (masked_template, value) in masked_templates.items() if value['mf'] == 'female'}
        male_masked_templates = {masked_template: value for (masked_template, value) in masked_templates.items() if value['mf'] == 'male'}

        female_filled_templates = [[fill['token_str'].strip() 
                                    for fill in nlp_fill(masked_sentence.replace('[M]', tokenizer.mask_token))] 
                                    for masked_sentence in female_masked_templates.keys()]
        male_filled_templates = [[fill['token_str'].strip() 
                                  for fill in nlp_fill(masked_sentence.replace('[M]', tokenizer.mask_token))] 
                                  for masked_sentence in male_masked_templates.keys()]
    #GPT2 generates next word directly
    else:
        # Define nlp_fill pipeline
        nlp_fill = pipeline('text-generation', model=model, tokenizer=tokenizer)

        # Fill templates (please check if the filled words contain any special character)
        female_masked_templates = {masked_template: value for (masked_template, value) in masked_templates.items() if value['mf'] == 'female'}

        male_masked_templates = {masked_template: value for (masked_template, value) in masked_templates.items() if value['mf'] == 'male'}
        # Tokenize the input into integer token IDs
        
        inputs_female = [tokenizer.encode(masked_sentence.replace(' [M].', ""), return_tensors='pt')  
                                    for masked_sentence in female_masked_templates.keys()]
        inputs_male = [tokenizer.encode(masked_sentence.replace(' [M].', ""), return_tensors='pt')  
                                    for masked_sentence in male_masked_templates.keys()]


        female_filled_sentences  = [model.generate(inputs,
                                                  Top_k =10,
                                                    do_sample=True,
                                                   repetition_penalty=1.5,
                                                   num_return_sequences = k,
                                                  max_length = inputs.shape[1]+1, pad_token_id=tokenizer.eos_token_id) for inputs in inputs_female]
        male_filled_sentences  = [model.generate(inputs,
                                              Top_k =10,
                                                do_sample=True,
                                               repetition_penalty=1.5,
                                               num_return_sequences = k,
                                              max_length = inputs.shape[1]+1, pad_token_id=tokenizer.eos_token_id) for inputs in inputs_male]

        female_filled_templates = [[tokenizer.decode(sent[-1]).strip() for sent in sents_k ] for sents_k in female_filled_sentences]
        male_filled_templates = [[tokenizer.decode(sent[-1]).strip() for sent in sents_k ] for sents_k in male_filled_sentences]

    female_honest_score = evaluator.honest(female_filled_templates)
    male_honest_score = evaluator.honest(male_filled_templates)
    print("Top_k:",k,"language:",lang)
    print("female honest score: ", female_honest_score)
    print("male honest score: ", male_honest_score)

## roberta-base BERT

In [98]:
# Load BERT model
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base")

In [106]:
for lang in ["en","es","fr","it","pt","ro"]:
    fill_in_and_evaluate_honest(lang, tokenizer=tokenizer, model=model,mask = True,k = 5)
    print("*"*50)

Top_k: 5 language: en
female honest score:  0.055491329479768786
male honest score:  0.049206349206349205
**************************************************
Top_k: 5 language: es
female honest score:  0.01282051282051282
male honest score:  0.07384615384615385
**************************************************
Top_k: 5 language: fr
female honest score:  0.02030456852791878
male honest score:  0.027692307692307693
**************************************************
Top_k: 5 language: it
female honest score:  0.03111111111111111
male honest score:  0.0009876543209876543
**************************************************
Top_k: 5 language: pt
female honest score:  0.0019656019656019656
male honest score:  0.017142857142857144
**************************************************
Top_k: 5 language: ro
female honest score:  0.002863961813842482
male honest score:  0.011428571428571429
**************************************************


## GPT2


In [108]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [110]:
for lang in ["en","es","fr","it","pt","ro"]:
    fill_in_and_evaluate_honest(lang, tokenizer=tokenizer, model=model,mask = False,k = 5)
    print("*"*50)

Top_k: 5 language: en
female honest score:  0.03757225433526012
male honest score:  0.03597883597883598
**************************************************
Top_k: 5 language: es
female honest score:  0.0
male honest score:  0.0
**************************************************
Top_k: 5 language: fr
female honest score:  0.018781725888324875
male honest score:  0.01641025641025641
**************************************************
Top_k: 5 language: it
female honest score:  0.0009876543209876543
male honest score:  0.0044444444444444444
**************************************************
Top_k: 5 language: pt
female honest score:  0.002457002457002457
male honest score:  0.005238095238095238
**************************************************
Top_k: 5 language: ro
female honest score:  0.0
male honest score:  0.0
**************************************************
